# Taller de Análisis de Datos - OnlineRetail.csv

## 1. Leer el archivo correctamente

Para el taller utilizarán el archivo: OnlineRetail.csv

In [1]:
import pandas as pd

df = pd.read_csv("../data/OnlineRetail.csv", encoding='latin-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 2. ¿Cuántos productos diferentes hay?

In [2]:
print("Cantidad de productos diferentes: ", df['Description'].nunique())

Cantidad de productos diferentes:  4223


## 3. ¿Cuántos clientes hay?

In [3]:
print("Cantidad de clientes: ", df['CustomerID'].nunique())

Cantidad de clientes:  4372


## 4. Identifique el país con mayor cantidad de facturas

In [4]:
facturas_por_pais = df.groupby('Country')['InvoiceNo'].nunique().sort_values(ascending=False)
print("País con mayor cantidad de facturas:", facturas_por_pais.index[0])


País con mayor cantidad de facturas: United Kingdom


## 5. Reporte a nivel de clientes

Preparar un reporte a nivel de clientes que incluya:
- La cantidad de facturas que tiene el cliente (hint: para hacer conteo único utilizamos nunique en lugar de count)
- El monto total comprado del cliente (hint: Revisar bien qué campo están usando👀)
- Cuándo fue su última compra

In [5]:
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']

# Agrupar por CustomerID y crear el reporte
reporte_clientes = df.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',  # Cantidad de facturas únicas
    'TotalAmount': 'sum',     # Monto total comprado
    'InvoiceDate': 'max'      # Última compra
}).reset_index()

# Renombrar columnas
reporte_clientes.columns = ['CustomerID', 'CantidadFacturas', 'MontoTotal', 'UltimaCompra']

print(reporte_clientes.head(10))

   CustomerID  CantidadFacturas  MontoTotal     UltimaCompra
0     12346.0                 2        0.00  1/18/2011 10:17
1     12347.0                 7     4310.00    8/2/2011 8:48
2     12348.0                 4     1797.24  9/25/2011 13:13
3     12349.0                 1     1757.55  11/21/2011 9:51
4     12350.0                 1      334.40   2/2/2011 16:01
5     12352.0                11     1545.41  9/28/2011 14:58
6     12353.0                 1       89.00  5/19/2011 17:47
7     12354.0                 1     1079.40  4/21/2011 13:11
8     12355.0                 1      459.40   5/9/2011 13:49
9     12356.0                 3     2811.43   4/8/2011 12:33


## 6. Filtrado y ordenamiento del reporte

Utilizando el reporte generado anteriormente:
- Filtrar dejando solo los clientes con un monto de compra mayor al promedio
- Y con una cantidad de facturas mayor a 1
- Ordenarlos de mayor a menor según el monto

In [8]:
monto_promedio = reporte_clientes['MontoTotal'].mean()

reporte_filtrado = reporte_clientes[
    (reporte_clientes['MontoTotal'] > monto_promedio) & 
    (reporte_clientes['CantidadFacturas'] > 1)
]

# Ordenar de mayor a menor según el monto
reporte_filtrado = reporte_filtrado.sort_values('MontoTotal', ascending=False)

print(f"Monto promedio: {monto_promedio.round(2)}")
print(f"\nClientes filtrados: {len(reporte_filtrado)}")
print("\nReporte filtrado y ordenado:")
print(reporte_filtrado.head(10))

Monto promedio: 1898.46

Clientes filtrados: 907

Reporte filtrado y ordenado:
      CustomerID  CantidadFacturas  MontoTotal     UltimaCompra
1703     14646.0                77   279489.02   9/8/2011 11:28
4233     18102.0                62   256438.49  9/28/2011 16:17
3758     17450.0                55   187482.17   9/5/2011 13:52
1895     14911.0               248   132572.62   9/9/2011 11:57
55       12415.0                26   123725.45   9/1/2011 13:51
1345     14156.0                66   113384.14   9/7/2011 16:00
3801     17511.0                46    88125.38   9/7/2011 11:41
3202     16684.0                31    65892.08  7/29/2011 15:12
1005     13694.0                60    62653.10  9/29/2011 10:22
2192     15311.0               118    59419.34   9/6/2011 14:38


## Bonus: 7. Pregunta personalizada

Formule y responda una pregunta que involucre al menos 2 conceptos aprendidos en pandas. (3 puntos adicionales)

In [7]:
# Pregunta: ¿Cuáles son los 5 productos más vendidos (por cantidad) en el país con mayor cantidad de facturas, 
# y cuál es el ingreso total generado por cada uno de estos productos?

# Realizamos filtrado, agrupación, ordenamiento y operaciones de agregación

# Paso 1: Filtrar por el país con mayor cantidad de facturas (United Kingdom)
df_uk = df[df['Country'] == 'United Kingdom']

# Paso 2: Agrupar por producto y calcular cantidad total vendida e ingreso total
productos_top = df_uk.groupby('Description').agg({
    'Quantity': 'sum',
    'TotalAmount': 'sum'
}).reset_index()

# Paso 3: Ordenar por cantidad y obtener el top 5
productos_top = productos_top.sort_values('Quantity', ascending=False).head(5)

# Renombrar columnas
productos_top.columns = ['Producto', 'CantidadVendida', 'IngresoTotal']

print(f"Top 5 productos más vendidos en {facturas_por_pais.index[0]}:\n")
print(productos_top.to_string(index=False))
print(f"\nIngreso total combinado de estos 5 productos: {productos_top['IngresoTotal'].sum()}")

Top 5 productos más vendidos en United Kingdom:

                          Producto  CantidadVendida  IngresoTotal
 WORLD WAR 2 GLIDERS ASSTD DESIGNS            48326      12055.96
           JUMBO BAG RED RETROSPOT            43167      84516.44
                    POPCORN HOLDER            34365      32425.81
     ASSORTED COLOUR BIRD ORNAMENT            33679      54662.15
WHITE HANGING HEART T-LIGHT HOLDER            33193      93953.07

Ingreso total combinado de estos 5 productos: 277613.43
